In [15]:

import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import json
from datetime import datetime, timezone

def weatherapi(latitude, longitude,location_name):
    # Setup session with caching and retry logic
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    client = openmeteo_requests.Client(session=retry_session)

    # API endpoint and parameters
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": [
            "temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature",
            "precipitation_probability", "precipitation", "rain", "showers", "snowfall",
            "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover","visibility",
            "evapotranspiration", "et0_fao_evapotranspiration", "vapour_pressure_deficit"
        ]
    }

    # Fetch weather data
    responses = client.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()

    # Generate datetime range for all hourly data points
    time_range = pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )

    # Find current UTC hour rounded down
    now_utc = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)

    # Find the index of the current hour in the time_range
    try:
        current_index = time_range.get_loc(now_utc)
    except KeyError:
        # If current time not in forecast range, fallback to nearest time (first available)
        current_index = 0

    # Prepare data for the current hour only
    current_hour_data = {"city_name": location_name}

    variables = [
        "temperature_celcius", "humidity_%", "dew_temperature_celcius", "feels_like_temperature_celcius",
        "precipitation_%", "precipitation_occured_mm", "rain_mm", "showers_mm", "snowfall_mm",
        "snow_depth_mm", "weather_code", "mean_sea_level_pressure_hpa", "surface_pressure_hpa", "cloud_cover_%", "visibility_m",
        "evapotranspiration_mm", "et0_fao_evapotranspiration_mm", "vapour_pressure_deficit_kpa"
    ]

    # Extract the value for the current hour from each variable
    for i, var in enumerate(variables):
        values = hourly.Variables(i).ValuesAsNumpy()
        current_hour_data[var] = float(values[current_index]) if values.size > current_index else None

    return current_hour_data

def fetch_weather_batch():
    delhi=weatherapi(28.7041,77.1025,'delhi')

    mumbai=weatherapi(18.9582,72.8321,'mumbai')

    bengaluru=weatherapi(12.9629,77.5775,'bengaluru')

    hyderabad=weatherapi(17.4065, 78.4772,'hyderabad')

    chennai=weatherapi(13.0843,80.2705,'chennai')

    kolkata=weatherapi(22.5744,88.3629,'kolkata')

    ahmedabad=weatherapi(23.0225,72.5714,'ahmedabad')

    pune=weatherapi(18.5246,73.8786,'pune')

    jaipur=weatherapi(26.9124,75.7873,'jaipur')

    lucknow=weatherapi(26.8467,80.9462,'lucknow')
    
    all_cities=[delhi,mumbai,bengaluru,hyderabad,chennai,kolkata,ahmedabad,pune,jaipur,lucknow]

    # debugging
    print(all_cities)


    return all_cities

fetch_weather_batch()











"""
    Fetches current hourly weather data for a specified location using the Open-Meteo API.
    
    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        location_name (str): Human-readable name of the location (for reference).
        
    Returns:
        dict: Weather data for the current hour, including temperature, humidity, precipitation,
              cloud cover, and other atmospheric variables.
              
    Implementation details:
        - Uses a cached session with automatic retries to improve API request reliability.
        - Fetches hourly forecast data from Open-Meteo API for a wide range of weather variables.
        - Converts API response timestamps to pandas datetime for easier time handling.
        - Extracts the data corresponding to the current UTC hour.
        - Returns a dictionary with all requested weather parameters for the current hour.

        
Data Dictionary Explanation:

- city_name: Name of the city (string).
- temperature_2m: Air temperature measured 2 meters above the ground, in degrees Celsius (float).
- relative_humidity_2m: Relative humidity at 2 meters height, in percentage (%) (float).
- dew_point_2m: Dew point temperature at 2 meters height, in degrees Celsius (float).
- apparent_temperature: Feels-like temperature considering humidity and wind, in degrees Celsius (float).
- precipitation_probability: Probability of precipitation occurring, in percentage (%) (float).
- precipitation: Amount of precipitation expected or recorded, in millimeters (mm) (float).
- rain: Amount of rain specifically, in millimeters (mm) (float).
- showers: Amount of showers, in millimeters (mm) (float).
- snowfall: Amount of snowfall, in millimeters (mm) (float).
- snow_depth: Depth of snow on the ground, in millimeters (mm) (float).
- weather_code: Numerical code representing weather conditions (e.g., clear, rain, snow) (float).
- pressure_msl: Atmospheric pressure at mean sea level, in hectopascals (hPa) (float).
- surface_pressure: Atmospheric pressure at the surface, in hectopascals (hPa) (float).
- cloud_cover: Total cloud cover percentage (%) (float).
- cloud_cover_low: Low-level cloud cover percentage (%) (float).
- cloud_cover_mid: Mid-level cloud cover percentage (%) (float).
- cloud_cover_high: High-level cloud cover percentage (%) (float).
- visibility: Visibility distance in meters (float).
- evapotranspiration: Amount of evapotranspiration, representing water transfer from land to atmosphere, in millimeters (mm) (float).
- et0_fao_evapotranspiration: Reference evapotranspiration (FAO standard), in millimeters (mm) (float).
- vapour_pressure_deficit: Difference between saturation and actual vapor pressure, indicating dryness of air, in kilopascals (kPa) (float).

"""


[{'city_name': 'delhi', 'temperature_celcius': 28.928499221801758, 'humidity_%': 87.0, 'dew_temperature_celcius': 26.54486083984375, 'feels_like_temperature_celcius': 36.12222671508789, 'precipitation_%': 20.0, 'precipitation_occured_mm': 2.5, 'rain_mm': 0.0, 'showers_mm': 2.5, 'snowfall_mm': 0.0, 'snow_depth_mm': 0.0, 'weather_code': 96.0, 'mean_sea_level_pressure_hpa': 996.9000244140625, 'surface_pressure_hpa': 972.2412109375, 'cloud_cover_%': 100.0, 'visibility_m': 24140.0, 'evapotranspiration_mm': 0.019999999552965164, 'et0_fao_evapotranspiration_mm': 0.0, 'vapour_pressure_deficit_kpa': 0.5179157257080078}, {'city_name': 'mumbai', 'temperature_celcius': 28.21500015258789, 'humidity_%': 88.0, 'dew_temperature_celcius': 26.036828994750977, 'feels_like_temperature_celcius': 31.667644500732422, 'precipitation_%': 88.0, 'precipitation_occured_mm': 0.6000000238418579, 'rain_mm': 0.0, 'showers_mm': 0.6000000238418579, 'snowfall_mm': 0.0, 'snow_depth_mm': 0.0, 'weather_code': 80.0, 'mean_s

'\n    Fetches current hourly weather data for a specified location using the Open-Meteo API.\n    \n    Args:\n        latitude (float): Latitude of the location.\n        longitude (float): Longitude of the location.\n        location_name (str): Human-readable name of the location (for reference).\n        \n    Returns:\n        dict: Weather data for the current hour, including temperature, humidity, precipitation,\n              cloud cover, and other atmospheric variables.\n              \n    Implementation details:\n        - Uses a cached session with automatic retries to improve API request reliability.\n        - Fetches hourly forecast data from Open-Meteo API for a wide range of weather variables.\n        - Converts API response timestamps to pandas datetime for easier time handling.\n        - Extracts the data corresponding to the current UTC hour.\n        - Returns a dictionary with all requested weather parameters for the current hour.\n\n        \nData Dictionary E

In [35]:
import pandas as pd
from datetime import datetime

#convert to dataframe
df = pd.DataFrame(fetch_weather_batch())

df = df.round({
    'temperature_celcius': 2,
    'humidity_%': 0,
    'dew_temperature_celcius': 2,
    'feels_like_temperature_celcius': 2,
    'precipitation_': 0,
    'precipitation_occured_mm': 2,
    'rain_mm': 2,
    'showers_mm': 2,
    'snowfall_mm': 2,
    'snow_depth_mm': 2,
    'mean_sea_level_pressure_hpa': 2,
    'surface_pressure_hpa': 1,
    'cloud_cover_%': 0,
    'visibility_m': 2,
    'evapotranspiration_mm': 3,
    'et0_fao_evapotranspiration_mm': 3,
    'vapour_pressure_deficit_kpa': 4
})


# Add current datetime in the specified format
df['created_at'] = datetime.now()

#add water strees index
df['water_stress_index'] = (
    df['evapotranspiration_mm'] / (df['precipitation_occured_mm'] + 0.01)  # Avoid div/0
).round(2)

#add column of check extreme condition or not
extreme_mask = (
    (df['temperature_celcius'] > 40) |
    (df['temperature_celcius'] < 0) |
    (df['snowfall_mm'] > 5) |
    (df['precipitation_occured_mm'] > 20) |
    (df['visibility_m'] < 500)
)
df['extreme_weather_yn'] = 'N'  # default value
df.loc[extreme_mask, 'extreme_weather_yn'] = 'Y'



[{'city_name': 'delhi', 'temperature_celcius': 28.07849884033203, 'humidity_%': 90.0, 'dew_temperature_celcius': 26.282512664794922, 'feels_like_temperature_celcius': 34.83643341064453, 'precipitation_%': 40.0, 'precipitation_occured_mm': 6.800000190734863, 'rain_mm': 0.0, 'showers_mm': 6.800000190734863, 'snowfall_mm': 0.0, 'snow_depth_mm': 0.0, 'weather_code': 95.0, 'mean_sea_level_pressure_hpa': 997.2999877929688, 'surface_pressure_hpa': 972.562744140625, 'cloud_cover_%': 100.0, 'visibility_m': 24140.0, 'evapotranspiration_mm': 0.05000000447034836, 'et0_fao_evapotranspiration_mm': 0.0, 'vapour_pressure_deficit_kpa': 0.3792533874511719}, {'city_name': 'mumbai', 'temperature_celcius': 28.26500129699707, 'humidity_%': 88.0, 'dew_temperature_celcius': 26.086030960083008, 'feels_like_temperature_celcius': 32.08738708496094, 'precipitation_%': 93.0, 'precipitation_occured_mm': 0.6000000238418579, 'rain_mm': 0.0, 'showers_mm': 0.6000000238418579, 'snowfall_mm': 0.0, 'snow_depth_mm': 0.0, '

In [36]:
import pandas as pd

# Show more columns
pd.set_option('display.max_columns', None)

# Optionally, show more rows (if you also need that)
pd.set_option('display.max_rows', None)

# Adjust column width (optional)
pd.set_option('display.max_colwidth', None)

# Prevent horizontal truncation
pd.set_option('display.width', None)  # Auto-detect terminal width

df.head(7)
# df.isnull().sum()
# df.dtypes

,city_name,temperature_celcius,humidity_%,dew_temperature_celcius,feels_like_temperature_celcius,precipitation_%,precipitation_occured_mm,rain_mm,showers_mm,snowfall_mm,snow_depth_mm,weather_code,mean_sea_level_pressure_hpa,surface_pressure_hpa,cloud_cover_%,visibility_m,evapotranspiration_mm,et0_fao_evapotranspiration_mm,vapour_pressure_deficit_kpa,created_at,water_stress_index,extreme_weather_yn
0,delhi,28.08,90.0,26.28,34.84,40.0,6.8,0.0,6.8,0.0,0.0,95.0,997.3,972.6,100.0,24140.0,0.05,0.000,0.3793,2025-06-25 03:50:27.809155,0.01,N
1,mumbai,28.27,88.0,26.09,32.09,93.0,0.6,0.0,0.6,0.0,0.0,80.0,1000.4,999.3,100.0,5060.0,0.13,0.038,0.4601,2025-06-25 03:50:27.809155,0.21,N
2,bengaluru,21.41,92.0,20.06,23.69,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1007.1,908.2,80.0,24140.0,0.02,0.000,0.2040,2025-06-25 03:50:27.809155,2.00,N
3,hyderabad,24.21,86.0,21.72,27.16,5.0,0.0,0.0,0.0,0.0,0.0,3.0,1002.5,944.9,100.0,24140.0,0.05,0.014,0.4229,2025-06-25 03:50:27.809155,5.00,N
4,chennai,27.23,72.0,21.74,30.92,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1002.6,1001.3,86.0,24140.0,0.06,0.026,1.0109,2025-06-25 03:50:27.809155,6.00,N
5,kolkata,27.36,93.0,26.12,34.10,38.0,0.1,0.0,0.1,0.0,0.0,3.0,998.6,997.5,100.0,24140.0,0.02,0.000,0.2545,2025-06-25 03:50:27.809155,0.18,N
6,ahmedabad,27.66,94.0,26.60,34.71,8.0,0.0,0.0,0.0,0.0,0.0,3.0,997.4,991.4,100.0,24140.0,0.02,0.000,0.2220,2025-06-25 03:50:27.809155,2.00,N
